In [ ]:
# !pip install ipywidgets
# !pip install matplotlib
# !pip install uk_covid19
# !pip install voila
# !pip install pandas

In [105]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import display
from uk_covid19 import Cov19API
import time

In [96]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

#**UK COVID 19 DASHBOARD**

*This website is developed to inform the genral public about the corona pandemic. It contains a graph that shows the number of cases reported and how many of them admitted to the hospital*

In [97]:
def wrangle_data(data):
    data = data['data']
#     print("data is ", data)
    dates = [dictionary['date'] for dictionary in data]
    dates.sort()

    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='d')
    df=pd.DataFrame(index=index, columns=['Cases','Admitted'])
    for entry in data:  # each entry is a dictionary with date, cases, hospital and deaths
        date = parse_date(entry['date'])
        for column in  ['Cases','Admitted']:
            if pd.isna(df.loc[date, column]):
                value = float(entry[column]) if entry[column] != None else 0.0
                df.loc[date, column] = value

    df.fillna(0.0, inplace=True)
    return df

In [98]:
def access_api():
    filters = {
        'areaType=overview',
    }
    structure = {
        "date": "date",
        "name":"areaName",
        "Cases": "newCasesByPublishDate",
        "Admitted": "hospitalCases"
    }
    api = Cov19API(filters=filters, structure=structure)
    return api.get_json()

In [99]:
def gather_data():
    with open("UK_stats.json", "wt") as Output:
        json.dump(access_api(), Output)

In [111]:
gather_data()
with open("UK_stats.json", "r") as t:
    data=json.load(t)
df=wrangle_data(data)

In [108]:
def main(button):
    refreshButton.button_style = "Primary"
    refreshButton.description = "Refreshing..."
    refreshButton.icon = ""
    data = gather_data()
    with open("UK_stats.json", "r") as t:
        data=json.load(t)
    global df
    df = wrangle_data(data)
    refresh_graph()
    refreshButton.button_style = "Success"
    refreshButton.description = "Refreshed"
    refreshButton.icon = "check"
    time.sleep(1)
    refreshButton.button_style = "info"
    refreshButton.description = "Refresh data"
    refreshButton.icon = "download"
    
refreshButton = wdg.Button(
    description='Refresh Data',  
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

refreshButton.on_click(main)  # the name of your function inside these brackets
display(refreshButton)


Button(button_style='info', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Keep ca…

##**Cases and Hospitalization accross UK per day**

In [110]:
def plot_random_walk(option):
    df[option].plot()
    
stats = wdg.Dropdown(
    options=['Cases', 'Admitted'],
    value='Cases',
    description='Stats: ',
    disabled=False,
)

    
def refresh_graph():
    current = stats.value
    if current == stats.options[0]:
        other = stats.options[1]
    else:
        other = stats.options[0]
    stats.value = other  
    stats.value = current  

    
graph = wdg.interactive_output(plot_random_walk, {'option': stats})
display(stats, graph)

Dropdown(description='Stats: ', options=('Cases', 'Admitted'), value='Cases')

Output()

The data is gathered from the [Public Health England](https://www.gov.uk/government/organisations/public-health-england) Website.

Copy Rights Reserved Muhammad Hassan Rasheed